In [1]:
import csv
import os
import os.path as osp
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MLP, global_add_pool, GraphConv

batch_size = 128
num_layers = [0, 1, 2, 3, 4, 5, 6]
lr = 0.001
epochs = 5
dataset_name_list = ["DD"]
num_reps = 5
hd = 64

color_counts = [
    [3, 231, 10416, 15208, 16029, 16450, 16722, 16895, 17026]
]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, nc):
        super().__init__()

        self.convs = torch.nn.ModuleList()
        for _ in range(nc):
            self.convs.append(GraphConv(in_channels, hidden_channels, aggr='add', bias=True))
            in_channels = hidden_channels

        if nc != 0:
            self.mlp = MLP([hidden_channels, hidden_channels, out_channels])
        else:
            self.mlp = MLP([in_channels, hidden_channels, out_channels])

    def forward(self, x, edge_index, batch):
        for conv in self.convs:
            x = torch.relu(conv(x, edge_index))
        x = global_add_pool(x, batch)

        return torch.sigmoid(self.mlp(x)) #return self.mlp(x) was before
    
    
    
for d, dataset_name in enumerate(dataset_name_list):
    path = osp.join(os.getcwd(), 'data', 'TU')
    dataset = TUDataset(path, name=dataset_name).shuffle()

    colors = sns.color_palette()

    raw_data = []
    table_data = []

    diffs = []
    diffs_std = []

    for l in num_layers:
        print(l)
        table_data.append([])
        for it in range(num_reps):

            dataset.shuffle()

            train_dataset = dataset[len(dataset) // 10:]
            train_loader = DataLoader(train_dataset, batch_size, shuffle=True)

            test_dataset = dataset[:len(dataset) // 10]
            test_loader = DataLoader(test_dataset, batch_size)

            model = Net(dataset.num_features, hd, dataset.num_classes, l).to(device)

            optimizer = torch.optim.Adam(model.parameters(), lr=lr)


            def train():
                model.train()

                total_loss = 0
                for data in train_loader:
                    data = data.to(device)
                    optimizer.zero_grad()
                    out = model(data.x, data.edge_index, data.batch)
                    loss = F.cross_entropy(out, data.y)
                    loss.backward()
                    optimizer.step()
                    total_loss += float(loss) * data.num_graphs
                return total_loss / len(train_loader.dataset)


            @torch.no_grad()
            def test(loader):
                model.eval()

                total_correct = 0
                for data in loader:
                    data = data.to(device)
                    pred = model(data.x, data.edge_index, data.batch).argmax(dim=-1)
                    total_correct += int((pred == data.y).sum())
                return total_correct / len(loader.dataset)


            for epoch in range(1, epochs + 1):
                loss = train()
                train_acc = test(train_loader) * 100.0
                test_acc = test(test_loader) * 100.0

            raw_data.append({'it': it, 'test': test_acc, 'train': train_acc, 'diff': train_acc - test_acc, 'layer': l,
                             'Color classes': color_counts[d][l]})

            table_data[-1].append([train_acc, test_acc, train_acc - test_acc, color_counts[d][l]])

    data = pd.DataFrame.from_records(raw_data)
    table_data = np.array(table_data)

    with open(dataset_name + '.csv', 'w') as file:
        writer = csv.writer(file, delimiter=' ', lineterminator='\n')

        for i, h in enumerate(num_layers):
            train = table_data[i][:, 0]
            test = table_data[i][:, 1]
            diff = table_data[i][:, 2]
            color = table_data[i][:, 3]

            writer.writerow([str(h)])
            writer.writerow(["###"])
            writer.writerow([train.mean(), train.std()])
            writer.writerow([test.mean(), test.std()])
            writer.writerow([diff.mean(), diff.std()])

            print(str(h))
            print("###")
            print(train.mean(), train.std())
            print(test.mean(), test.std())
            print(diff.mean(), diff.std())
            print(color[-1])

c:\Users\Aneeqa\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
c:\Users\Aneeqa\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


0
1
2
3
4
5
6
0
###
74.32610744580583 0.9032345749829191
75.8974358974359 1.133888817215522
-1.5713284516300576 0.6118925417991246
3.0
1
###
73.21394910461828 1.1456792760808856
73.50427350427351 1.4301880795454283
-0.29032439965523055 0.32179200355854126
231.0
2
###
71.61168708765317 2.015643422594276
72.64957264957266 1.8725557521544178
-1.0378855619194922 1.0297309095353373
10416.0
3
###
72.0640904806786 4.245092015619881
73.5042735042735 3.6261886214694754
-1.4401830235949062 1.7155674779209116
15208.0
4
###
71.63053722902922 2.9750993668692427
72.991452991453 1.7599367762370937
-1.3609157624237724 1.9730630315380793
16029.0
5
###
67.38925541941565 6.330673418692093
69.74358974358975 6.89293851407041
-2.3543343241740926 1.5575799272480577
16450.0
6
###
71.14043355325164 6.665970780013746
70.5982905982906 6.58951633073381
0.5421429549610423 2.588705150820402
16722.0
